# 🧩 Mobile Price 예측 모델링 (회귀 과제 편)
- 빅데이터 분석기사 실기 연습을 위해 제가 만든 자료입니다.
- 데이터셋은 kaggle의 데이터셋을 제가 다시 가공한 뒤 X_train, y_train, X_test로 분리하였습니다. 원본 데이터셋은 다음의 주소에서 확인하실 수 있습니다 : (https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification/data?select=train.csv) 

### 🧩 문제
- 주어진 학습용 데이터를 활용하여 모바일폰의 ram의 용량을 예측하는 모형을 만들고, 이를 평가용 데이터인 X_test.csv에 적용하여 ram의 예측값을 csv 파일로 생성하시오.(평가 지표는 RMSE) 

### 🧩 힌트
- 분류 과제인지 회귀 과제인지 판단합니다.
- 평가지표가 RMSE인 것으로 보아 회귀 과제에 해당합니다.
- 회귀 과제라면 proba가 아닌 pred가 사용됩니다. 회귀과제는 클래스에 속할 확률을 예측하는 것이 아니기 때문에 proba가 사용되지 않습니다.

### 🧩 주의
- 이 모델은 rmse 수치가 매우 높습니다. 자체 가공한 데이터셋으로 타겟을 ram으로 설정한 것이 적절하지 않았을 수 있습니다. 회귀 과제를 연습하는 방향으로만 사용하는 것이 좋겠습니다.

# 1. 필요한 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# 2. 데이터 파일 읽기
- 시험에는 총 3개의 파일이 주어집니다.
- X_train : 학습에 사용되는 변수들이 포함된 데이터입니다.
- X_test : 평가에 사용되는 데이터입니다.
- y_train : 타겟 데이터입니다. 여기서는 ram의 용량을 나타냅니다.

In [2]:
X_train = pd.read_csv('./Reg_Mobile_X_train.csv')
X_test = pd.read_csv('./Reg_Mobile_X_test.csv')
y_train = pd.read_csv('./Reg_Mobile_y_train.csv')

In [3]:
X_train.head(2)

,battery_power,clock_speed,fc,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,sc_h,sc_w,wifi
0,842.0,2.2,1,7,0.6,188.0,2,2,20,756,9,7,yes
1,1021.0,0.5,0,53,0.7,136.0,3,6,905,1988,17,3,no


In [4]:
y_train.head(2)

,ram
0,2549
1,2631


# 3. info() 함수로 기본 정보 확인하기
- 행과 열의 개수 및 결측치가 있는지 알아봅니다.
- 각 열의 데이터 타입을 확인하여 이후 수치형 변수와 명목형 변수로 나눌 때 참고합니다.
- X_train에는 3개의 열에 결측치가 존재합니다.
- X_train은 명목형 변수(object) 컬럼이 하나 존재합니다.

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1998 non-null   float64
 1   clock_speed    2000 non-null   float64
 2   fc             2000 non-null   int64  
 3   int_memory     2000 non-null   int64  
 4   m_dep          2000 non-null   float64
 5   mobile_wt      1998 non-null   float64
 6   n_cores        2000 non-null   int64  
 7   pc             2000 non-null   int64  
 8   px_height      2000 non-null   int64  
 9   px_width       2000 non-null   int64  
 10  sc_h           2000 non-null   int64  
 11  sc_w           2000 non-null   int64  
 12  wifi           1997 non-null   object 
dtypes: float64(4), int64(8), object(1)
memory usage: 203.2+ KB


In [6]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1000 non-null   int64  
 1   clock_speed    1000 non-null   float64
 2   fc             1000 non-null   int64  
 3   int_memory     1000 non-null   int64  
 4   m_dep          1000 non-null   float64
 5   mobile_wt      1000 non-null   int64  
 6   n_cores        1000 non-null   int64  
 7   pc             1000 non-null   int64  
 8   px_height      1000 non-null   int64  
 9   px_width       1000 non-null   int64  
 10  sc_h           1000 non-null   int64  
 11  sc_w           1000 non-null   int64  
 12  wifi           1000 non-null   object 
dtypes: float64(2), int64(10), object(1)
memory usage: 101.7+ KB


In [7]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ram     1998 non-null   int64
dtypes: int64(1)
memory usage: 15.7 KB


# 4. 결측치 처리하기
- X_train과 y_train에 결측치가 모두 존재하기 때문에 이를 고려하여 결측치를 처리하는 방법을 선정합니다.
- 두 데이터셋의 결측치 행을 확인합니다.

In [8]:
train = pd.concat([X_train, y_train], axis = 1)
train.head(1)

,battery_power,clock_speed,fc,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,sc_h,sc_w,wifi,ram
0,842.0,2.2,1,7,0.6,188.0,2,2,20,756,9,7,yes,2549.0


In [9]:
missing_data_rows = train[train.isnull().any(axis=1)]
missing_data_rows            

,battery_power,clock_speed,fc,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,sc_h,sc_w,wifi,ram
19,682.0,0.5,4,19,1.0,121.0,4,11,902,1064,11,1,NaN,2337.0
20,NaN,1.1,12,39,0.8,NaN,7,14,1314,1854,17,15,NaN,1433.0
21,NaN,2.1,1,13,1.0,NaN,2,2,974,1385,17,1,NaN,1037.0
1998,1512.0,0.9,4,46,0.1,145.0,5,5,336,670,18,10,yes,NaN
1999,510.0,2.0,5,45,0.9,168.0,6,16,483,754,19,4,yes,NaN


- 결측치가 컬럼마다 여러 행에 분포되어 있는 상황이므로 dropna() 함수를 사용하여 결측치가 포함된 행들을 제거합니다.
- 결측치를 제거한 뒤에 다시 X_train과 y_train으로 나누어 모델링을 진행합니다.

In [10]:
train = train.dropna()
train.reset_index(drop = True, inplace = True)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995 entries, 0 to 1994
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1995 non-null   float64
 1   clock_speed    1995 non-null   float64
 2   fc             1995 non-null   int64  
 3   int_memory     1995 non-null   int64  
 4   m_dep          1995 non-null   float64
 5   mobile_wt      1995 non-null   float64
 6   n_cores        1995 non-null   int64  
 7   pc             1995 non-null   int64  
 8   px_height      1995 non-null   int64  
 9   px_width       1995 non-null   int64  
 10  sc_h           1995 non-null   int64  
 11  sc_w           1995 non-null   int64  
 12  wifi           1995 non-null   object 
 13  ram            1995 non-null   float64
dtypes: float64(5), int64(8), object(1)
memory usage: 218.3+ KB


In [12]:
X_train = train[['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
                'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'sc_h', 'sc_w', 'wifi']]
y_train = train[['ram']]

In [13]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995 entries, 0 to 1994
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ram     1995 non-null   float64
dtypes: float64(1)
memory usage: 15.7 KB


# 5. describe() 함수로 요약 통계량 확인하기

In [14]:
X_train.describe()

,battery_power,clock_speed,fc,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,sc_h,sc_w
count,1995.000000,1995.000000,1995.000000,1995.000000,1995.000000,1995.00000,1995.000000,1995.000000,1995.000000,1995.000000,1995.000000,1995.000000
mean,1239.023559,1.522757,4.307268,32.045614,0.501103,140.26416,4.519799,9.917293,644.715288,1251.781454,12.296241,5.765915
std,439.195995,0.816359,4.342799,18.155420,0.287989,35.40917,2.289040,6.066059,443.916960,432.160104,4.211195,4.353124
min,501.000000,0.500000,0.000000,2.000000,0.100000,80.00000,1.000000,0.000000,0.000000,500.000000,5.000000,0.000000
25%,852.000000,0.700000,1.000000,16.000000,0.200000,109.00000,3.000000,5.000000,282.000000,875.500000,9.000000,2.000000
50%,1227.000000,1.500000,3.000000,32.000000,0.500000,141.00000,4.000000,10.000000,564.000000,1247.000000,12.000000,5.000000
75%,1615.500000,2.200000,7.000000,48.000000,0.800000,170.00000,7.000000,15.000000,946.500000,1633.000000,16.000000,9.000000
max,1998.000000,3.000000,19.000000,64.000000,1.000000,200.00000,8.000000,20.000000,1960.000000,1998.000000,19.000000,18.000000


- X_train에서 특별한 이상치는 발견되지 않는 것으로 판단됩니다.

# 6. 수치형 / 명목형 변수 정리, 명목형 변수는 레이블 인코딩

- 회귀모델은 입력 데이터가 수치형일 때 잘 작동됩니다. 모델이 명목형 변수를 처리할 수 있도록 숫자 값으로 변환해야 이를 또 하나의 변수로 사용하여 모델을 학습시킬 수 있습니다.
- 명목형 변수를 처리하는 방법으로 원-핫 인코딩이 많이 사용됩니다.

In [15]:
col_number = ['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
                'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'sc_h', 'sc_w']
col_category = ['wifi']
col_y = ['ram']

- 훈련 데이터와 테스트 데이터 모두 동일한 원-핫 인코딩 방식을 적용하기 위해 X_train와 X_test를 concat해서 피팅시키고, 훈련 데이터와 테스트 데이터에 각각 인코딩을 진행합니다.
- 원-핫 인코딩은 범주 간에 순서가 없습니다. 명목형 변수에서 각 범주의 데이터를 0, 1, 2 등으로 정수로 바꾸는 레이블 인코딩은 간단할 수도 있지만 정수 0, 1, 2 사이에는 순서가 존재하는 것(더 큰 값)으로 가정될 수 있습니다. 따라서 범주 간에 순서가 존재하지 않는다면 원-핫 인코딩이 더 적절합니다. 

In [16]:
X = pd.concat([X_train, X_test])

In [17]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
onehot.fit(X[col_category])

X_train_res = onehot.transform(X_train[col_category])
X_test_res = onehot.transform(X_test[col_category])

- .todense() 메서드는 이 희소 행렬을 일반적인 밀집 행렬(dense matrix)로 변환합니다. 이는 Pandas 데이터프레임으로 쉽게 변환할 수 있는 형태입니다.
- onehot.get_feature_names()는 원-핫 인코딩된 특성의 이름을 가져옵니다.

In [18]:
X_train_onehot = pd.DataFrame(X_train_res.todense(), columns = onehot.get_feature_names())
X_test_onehot = pd.DataFrame(X_test_res.todense(), columns = onehot.get_feature_names())
print(X_train_onehot)

      x0_no  x0_yes
0       0.0     1.0
1       1.0     0.0
2       1.0     0.0
3       1.0     0.0
4       1.0     0.0
...     ...     ...
1990    0.0     1.0
1991    1.0     0.0
1992    1.0     0.0
1993    0.0     1.0
1994    1.0     0.0

[1995 rows x 2 columns]


In [19]:
X_train_last = pd.concat([X_train[col_number], X_train_onehot], axis=1)
X_test_last = pd.concat([X_test[col_number], X_test_onehot], axis=1)

# 7. 학습용 데이터와 검증용 데이터 분할
- 학습 데이터로 만든 모델의 일반화 성능을 검증하기 위해 다시 분할하여 검증합니다.
- 과적합이 발생하는 경우에 모델을 바꾸거나 하이퍼파라미터를 조정하는 등의 방식으로 모델링 방식을 바꿉니다. 
- 여기서 학습 데이터와 검증 데이터를 나누는 비율은 0.3으로 설정하였습니다.

In [20]:
from sklearn.model_selection import train_test_split

X_tr, X_valid, y_tr, y_valid = train_test_split(X_train_last, y_train, test_size = 0.3)

# 8. 스케일링 수행

In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_tr[col_number])

X_tr[col_number] = scaler.transform(X_tr[col_number])
X_valid[col_number] = scaler.transform(X_valid[col_number])
X_test_last[col_number] = scaler.transform(X_test_last[col_number])

# 9. 모델 학습
- 랜덤 포레스트 모델을 사용합니다. 분류 과제에서는 RandomForestClassifier를 사용한 반면에 회귀 과제에서는 RandomForestRegressor을 사용합니다.

In [22]:
from sklearn.ensemble import RandomForestRegressor

modelRF = RandomForestRegressor(n_estimators=100, random_state=42)
modelRF.fit(X_tr, y_tr)

RandomForestRegressor(random_state=42)

# 10. 모델 평가
- 이 모델은 rmse 수치가 매우 높습니다. 자체 가공한 데이터셋으로 타겟을 ram으로 설정한 것이 적절하지 않았을 수 있습니다. 회귀 과제를 연습하는 방향으로만 사용하는 것이 좋겠습니다.

In [23]:
y_validation_predict = modelRF.predict(X_valid)

In [24]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_valid, y_validation_predict, squared=False)
print(rmse)

1134.9055638573886


- 스케일링을 진행한 최종 평가용 데이터에 모델을 적용하여 예측값을 출력합니다.

In [25]:
pred = modelRF.predict(X_test_last)
pred

array([1710.98, 2544.75, 2144.03, 2147.14, 2285.75, 2301.04, 2398.92,
       1955.27, 2294.22, 2131.37, 2444.47, 1849.07, 2122.98, 2370.34,
       2189.33, 2088.25, 2020.71, 2650.14, 2179.31, 2470.5 , 1903.01,
       2353.16, 2181.3 , 2269.03, 2154.5 , 2091.48, 2109.66, 2182.47,
       2175.16, 2283.74, 2159.75, 2035.09, 1990.36, 2218.89, 2098.64,
       2084.38, 1925.19, 2124.17, 2684.46, 2096.07, 1675.03, 2783.45,
       2470.34, 2203.28, 1918.6 , 2356.43, 2313.53, 2211.24, 2026.62,
       2234.49, 2093.09, 2437.51, 2136.82, 2489.3 , 2036.94, 2385.34,
       1495.05, 1957.6 , 2077.44, 2004.64, 2207.87, 2444.55, 2311.46,
       2614.55, 2434.17, 2229.4 , 1841.29, 2267.12, 2037.53, 2180.22,
       2716.85, 2211.91, 2252.42, 2140.45, 1641.86, 1792.69, 2221.65,
       2048.67, 2271.63, 1958.93, 2398.68, 1926.71, 1754.99, 2125.92,
       2137.44, 1871.53, 1932.66, 2420.98, 2381.07, 2350.26, 2003.47,
       2097.57, 2169.03, 2178.36, 2031.01, 2081.91, 1834.59, 1730.34,
       2057.14, 2129

# 11. 파일 제출

In [26]:
pd.DataFrame({'submission' : pred}).to_csv('regression_submission.csv', index = False)

In [27]:
df = pd.read_csv('./regression_submission.csv')
df

,submission
0,1710.98
1,2544.75
2,2144.03
3,2147.14
4,2285.75
...,...
995,2113.52
996,2186.91
997,2464.80
998,2418.25
